
<img src="img/viu_logo.png" width="200">

## 01MIAR - Procesamiento de Datos

![logo](img/python_logo.png)

*Ivan Fuertes*

# Sumario
- Trabajo con strings
- Combinando datasets
- Limpieza de datos
 - map, filter, reduce
 - filling missing values
 - valores duplicados
 - categorizacion de datos

In [1]:
import numpy as np
import pandas as pd

### Combinar varios datasets 
- En base a un elemento en común (índice)
- MovieLens 'UserId'

In [2]:
import zipfile as zp # para descomprimir archivos zip
import urllib.request # para descargar de URL
import os

# descargar MovieLens dataset
url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'  
local_zip = os.path.join("res", "ml-1m.zip")
urllib.request.urlretrieve(url, local_zip)
# descomprimiendo archivo zip
with zp.ZipFile(local_zip, 'r') as zipp: 
    print('Extracting all files...') 
    zipp.extractall(os.path.join("res")) # destino
    print('Done!') 

Extracting all files...
Done!


In [3]:
root_path = os.path.join("res", "ml-1m" )

ratings_dataset = pd.read_csv(os.path.join(root_path, "ratings.dat"), sep='::',
                                index_col=0, engine='python', encoding="ISO-8859-1",
                                names=['UserID','MovieID','Rating','Timestamp'])

users_dataset = pd.read_csv(os.path.join(root_path, "users.dat"),sep='::',
                              index_col=0, engine='python',  encoding="ISO-8859-1",
                              names=['UserID','Gender','Age','Occupation','Zip-code'])

In [4]:
display(users_dataset.sample(5))

,Gender,Age,Occupation,Zip-code
UserID,,,,
2872,M,25,20,94014
1826,M,45,11,49265
159,F,45,0,37922
576,F,45,15,20910
5232,M,25,4,14607


In [5]:
display(ratings_dataset.sample(5))

,MovieID,Rating,Timestamp
UserID,,,
1745,3420,4,974704837
2859,50,5,973128983
2680,1270,4,973389466
3618,1077,2,967121168
5664,457,5,958849121


### Uniendo datasets con 'join' y 'merge'
- merge() == join()
 - 'join' utiliza por defecto los índices para unir
- Utilizando el parámetro 'on'
 - Si las columnas difieren, 'left_on' y 'right_on'
 
 https://i.stack.imgur.com/hMKKt.jpg

In [6]:
# combinando users y ratings, ¿Cómo?
combined_dataset = users_dataset.merge(ratings_dataset, on='UserID', how='inner') # parametro 'on' define la columna pivote
display(combined_dataset.head(5))
print(len(combined_dataset))

,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp
UserID,,,,,,,
1,F,1,10,48067,1193,5,978300760
1,F,1,10,48067,661,3,978302109
1,F,1,10,48067,914,3,978301968
1,F,1,10,48067,3408,4,978300275
1,F,1,10,48067,2355,5,978824291


1000209


In [7]:
movies_dataset = pd.read_csv(os.path.join(root_path, "movies.dat"),sep='::', engine='python', encoding="ISO-8859-1", names=['MovieID','Title','Genre'])
display(movies_dataset.sample(5))

,MovieID,Title,Genre
770,780,Independence Day (ID4) (1996),Action|Sci-Fi|War
1991,2060,BASEketball (1998),Comedy
3278,3347,Never Cry Wolf (1983),Drama
2578,2647,House of Frankenstein (1944),Horror
2218,2287,Them! (1954),Sci-Fi|Thriller|War


In [8]:
# combinando movies y el resto
all_dataset = combined_dataset.merge(movies_dataset,on='MovieID', how='inner')
display(all_dataset.head(5))

,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Title,Genre
0,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


### Concatenate
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

## Pivot
- Representar los datos en función a varios parámetros, agregando
```python
pivot_table(<lista de valores>, index=<agregador primario>, columns=<agregador secundario>)
```
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html
- https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

In [9]:
display(all_dataset.pivot_table('Rating', index='Gender', columns='Age'))
display(all_dataset.pivot_table('Rating', index='Gender', columns='Age', aggfunc='count'))
display(all_dataset.pivot_table('Rating', index='Gender', columns='Age', aggfunc=['count', 'mean']))

Age,1,18,25,35,45,50,56
Gender,,,,,,,
F,3.616291,3.453145,3.60670,3.659653,3.663044,3.797110,3.915534
M,3.517461,3.525476,3.52678,3.604434,3.627942,3.687098,3.720327


Age,1,18,25,35,45,50,56
Gender,,,,,,,
F,8827,45427,91340,49473,24110,18064,9199
M,18384,138109,304216,149530,59523,54426,29581


count                                                   mean  \
Age        1       18      25      35     45     50     56        1    
Gender                                                                 
F        8827   45427   91340   49473  24110  18064   9199  3.616291   
M       18384  138109  304216  149530  59523  54426  29581  3.517461   

                                                                   
Age           18       25        35        45        50        56  
Gender                                                             
F       3.453145  3.60670  3.659653  3.663044  3.797110  3.915534  
M       3.525476  3.52678  3.604434  3.627942  3.687098  3.720327

## Agrupaciones
- agg -> funciones estadísticas de agregación
- Series.unique() -> valores únicos
- pd.value_counts -> ocurrencias

## Manipulación de strings
```python
split(): separar en bloques en función de un carácter
replace(): reemplazar un carácter por otro
index(): encontrar la posición de un carácter
```

In [10]:
# Ejemplo con MovieLens: Genre
## 1: obtener todos los géneros por separado
## 2: crear un dataset de géneros
## 3: por película, marcar género por separado
## 4: unir con dataset original
display(movies_dataset.head(3))

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [11]:
all_genres = movies_dataset['Genre'].apply(lambda x : x.split('|'))
print(all_genres)

# print([genre for x in all_genres for genre in x])

# genres = pd.unique([genre for movie in all_genres for genre in movie])

genres = pd.unique(all_genres.sum())
print(genres)

0        [Animation, Children's, Comedy]
1       [Adventure, Children's, Fantasy]
2                      [Comedy, Romance]
3                        [Comedy, Drama]
4                               [Comedy]
                      ...               
3878                            [Comedy]
3879                             [Drama]
3880                             [Drama]
3881                             [Drama]
3882                   [Drama, Thriller]
Name: Genre, Length: 3883, dtype: object
['Animation' "Children's" 'Comedy' 'Adventure' 'Fantasy' 'Romance' 'Drama'
 'Action' 'Crime' 'Thriller' 'Horror' 'Sci-Fi' 'Documentary' 'War'
 'Musical' 'Mystery' 'Film-Noir' 'Western']


In [12]:
# crear tabla con columnas por género
zeros = np.zeros( (len(movies_dataset), len(genres)) )
genres_frame = pd.DataFrame(zeros, columns=genres)
display(genres_frame.head(3))

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
columns_genres = genres_frame.columns # lista de generos (columnas)
# para cada película, marcar género con 1
for i, genre in enumerate(movies_dataset['Genre']):
    inds = columns_genres.get_indexer(genre.split('|')) # retorna los indices correspondientes a los generos de cada pelicula
    genres_frame.iloc[i,inds] = 1 # localiza las columnas del genero correspondiente, marca con 1

In [14]:
for i, genre in enumerate(movies_dataset['Genre']):
    genres_frame.loc[i, genre.split('|')] = 1

In [15]:
display(genres_frame.head(5))

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# unir con dataset original
movies_split_genre = movies_dataset.join(genres_frame)

In [ ]:
display(movies_split_genre.head(5))

#### Replace e index para extraer el año de la película

In [ ]:
display(movies_dataset.head(2))

In [ ]:
# extraer el año de la columna Title
def split_year(title):
    index = title.index('(')
    return title[index:].replace('(','').replace(')','')
    
# crear nueva columna Year
movies_dataset['Year'] = movies_dataset['Title'].apply(split_year)
display(movies_dataset.sample(2))

In [ ]:
# eliminar el año de la columna Title
def remove_year(title):
    index = title.index('(')
    return title[:index-1].strip()

movies_dataset['Title'] = movies_dataset['Title'].apply(remove_year)
display(movies_dataset.head(2))

#### Expresiones regulares
https://docs.python.org/3/library/re.html

- import re

In [ ]:
# ¿Cómo localizar que 'Zip-code' tiene un formato erróneo?
users_dataset.sample(5)

# users_dataset['Zip-code'].str.match('^[0-9]{5}$')

display(users_dataset[users_dataset['Zip-code'].str.match('^\d{5}$') == False])

# ^\d{5}$
# ^ = start of the string
# \d = decimal string
# {5} = 5 repeticiones de decimales
# $ = end of string

In [ ]:
movies_dataset = pd.read_csv(os.path.join(root_path, "movies.dat"),sep='::', engine='python', encoding='ISO-8859-1', names=['MovieID','Title','Genre'])
display(movies_dataset.head(2))

In [ ]:
# ¿Cómo extraer el año con regex en el formato adecuado?
display(movies_dataset['Title'].str.extract('(\d{4})'))

# (\d{4})
# (= busca apertura parentesis
# \d = decimal string
# {4} = 4 repeticiones de decimales
# ) = cierre de parentesis

## Operaciones con colecciones
```python
reduce: aplicar una operación y retornar un valor
map: aplicar  una operación y retornar una secuencia
filter: retorna una secuencia con elementos que cumplen una condición
```


## Reduce
- Aplicar una operación matemática a cada uno de los elementos de una colección
- Diferente de 'apply()' porque retorna un valor numérico
- Ejemplo: Detección de géneros en años específicos

https://docs.python.org/3/library/functools.html

In [1]:
from functools import reduce # necesario para reduce

lista = [1, 3, 5, 7, 9]
print(reduce(lambda x,y: x + y, lista))

25


In [ ]:
movies_1975 = movies_split_genre[ movies_split_genre['Title'].str.contains('1975') ]
movies_1975.head(3)

In [ ]:
any_drama = reduce(lambda x,y : bool(x) | bool(y),movies_1975['Drama']) # hay algún drama en 1975
print(any_drama)

all_comedy = reduce(lambda x,y : bool(x) & bool(y),movies_1975['Comedy']) # son todas las películas de 1975 comedias?
print(all_comedy)

In [ ]:
print(movies_1975['Drama'].any()) # Comprueba si hay algún valor que puede cumplir  
print(movies_1975['Comedy'].all()) # Comprueba si todos los valores son True

In [ ]:
# Observar el tipo de dato antes para ver si es posible aplicar las funciones
print(movies_1975.dtypes)
print(movies_1975['Comedy'].unique())

## Filter
- retorna una secuencia con elementos que cumplen una condición
- Ejemplo: obtener las películas de 1975 que contienen 'The' en el título

In [1]:
filtro = filter(lambda x : 'The' in x, movies_1975['Title'])
list(filtro)
# ¿Están todos los títulos con "The"? si tiene mayúsculas o no...

NameError: name 'movies_1975' is not defined

## Map
- aplicar  una operación y retornar una secuencia
- Cambiar el valor integral de la columna 'Comedy' por bool

In [ ]:
mapa = map(lambda x : bool(x), movies_split_genre['Comedy'])
movies_split_genre.loc[:,'Comedy'] = list(mapa)
display(movies_split_genre.head(4))

## Transformación de variables (calidad de datos)
- Valores no definidos
- Valores duplicados
- Discretización (valores categóricos)

In [ ]:
matrix = pd.DataFrame(np.random.randint(10,size=(5,10)))
matrix[matrix < 2] = np.nan
display(matrix)

In [ ]:
# nulos por columna
print(matrix.isnull().sum(axis=1))
# display(matrix.isna().sum())

In [ ]:
# Cantidad valores nulos
print(matrix.isnull().sum(axis=1).sum())

In [ ]:
# numero de no nulos por fila
print(matrix.count(axis=1))

In [ ]:
# Número de nulos por fila
print(matrix.shape[1] - matrix.count(axis=1))

In [ ]:
# Representación de las filas en las que una determinada columna tiene nulos
display(matrix[matrix[0].isnull()])

In [ ]:
# Conteo de valores que aparecen en el dataset
valores = [8, 4]
# Identificación de valores de dominio que se encuentran en un listado
display(matrix[matrix[6].isin(valores)])

In [ ]:
## Tratamiento de valores nulos
# eliminar
display(matrix.dropna(axis=1))

In [ ]:
# eliminar si no hay un número de valores no NaN
display(matrix)
display(matrix.dropna(thresh=7))

In [ ]:
# sustituir por un valor fijo
display(matrix.fillna(-1))

In [ ]:
# sustituir por valor dinámico (copia)
display(matrix)
display(matrix.fillna(method='bfill')) # bfill y ffill

In [ ]:
# sustituir por valor dinámico (interpolación)
display(matrix)
display(matrix.interpolate())

#### Tratar valores duplicados

In [ ]:
serie = pd.Series(['a','b','c','a','c','a','g'])
print(serie.duplicated())

In [ ]:
df = all_dataset
display(df.head(3))

# Eliminación de los duplicados en una columna definida
df2 = df.drop_duplicates(subset="Gender", keep='last', inplace=False)
display(df2)

#### Discretización (valores categóricos)
- Tras Series y DataFrame, objeto para categorías: Categorical
```python
categorias = pd.cut(<valores>, <bins>) 
```

In [ ]:
# especificar los bloques
bins = [0,18,35,65,99]
edades = [16,25,18,71,44,100,12]
categorias = pd.cut(edades,bins)
print(categorias)

In [ ]:
categorias.value_counts()

In [ ]:
# especificar el número de bloques
bins = 5
edades = [0,6,8,16,25,18,71,44,100]
categorias = pd.cut(edades,bins) # rangos idénticos (similar distancia de rangos)
print(categorias)
print(categorias.value_counts())

In [ ]:
bins = 5
edades = [1,6,8,16,25,18,71,44,100]
categorias = pd.qcut(edades,bins) # rangos homogéneos (similar número de valores)
print(categorias)
print(categorias.value_counts())